In [27]:
from dotenv import load_dotenv

env = load_dotenv()

from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("[PJT] ParsingOutput")

LangSmith 추적을 시작합니다.
[프로젝트명]
[PJT] ParsingOutput


In [28]:
email_conversation = """from:테디 (teddy@teddynote.com)
To: 이은채 대리님(eunchae@teddyinternational.me)
Subject: RAG 솔루션 시연 관련 미팅 제안

안녕하세요, 이은채 대리님,

저는 테디노트의 테디입니다. 최근 귀사에서 AI를 활용한 혁신적인 솔루션을 모색 중이라는 소식을 들었습니다. 테디노트는 AI 및 RAG 솔루션 분야에서 다양한 경험과 전문성을 가지고 있습니다.

저희 테디노트의 RAG 솔루션은 귀사의 데이터 활용을 극대화하고, 실시간으로 정확한 정보 제공을 통해 비즈니스 의사결정을 지원하는데 탁월한 성능을 보입니다.

이 기회에 귀사와의 협력 가능성을 논의하고, 저희 RAG 솔루션의 구체적인 기능과 적용 방안을 시연하기 위해 미팅을 제안드립니다. 다음 주 목요일(7월 18일) 오전 10시에 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

미팅 시간을 조율하기 어려우시다면, 편하신 다른 일정을 알려주시면 감사하겠습니다. 이은채 대리님과의 소중한 만남을 통해 상호 발전적인 논의가 이루어지길 기대합니다.

감사합니다.

테디
테디노트 AI 솔루션팀"""

In [29]:
## LCEL 구조


from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [30]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [31]:
# 이메일 본문에서 주요 엔티티 추출하기
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    company_info: str = Field(description="메일을 보낸 사람의 회사")
    phone_number: str = Field(description="메일 본문에 언급된 전화번호")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [32]:
from langchain_core.output_parsers import PydanticOutputParser

# PydanticOutputParser 생성
output_parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [33]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

#QUESTION:
다음의 이메일 내용 중에서 주요 내용을 추출해 주세요

#EMAIL CONVERSATION:
{email_conversation}

#FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=output_parser.get_format_instructions())

In [34]:
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"title": "Person", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c", "type": "string"}, "company_info": {"title": "Company Info", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c\uc758 \ud68c\uc0ac", "type": "string"}, "phone_number": {"title": "Phone Number", "description": "\uba54\uc77c \ubcf8\ubb38\uc5d0 \uc5b8\uae09\ub41c \uc804\ud654\ubc88\ud638", "type": "string"}, "email": {"title": "Email", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c\uc758 \uc774\uba54\uc77c \uc8fc\uc

In [35]:
# 체인생성
# prompt 를 만들려면 output_parser 의 format 이 필요함
chain = prompt | llm | output_parser

In [36]:
answer = chain.invoke({"email_conversation": email_conversation})

In [37]:
print(answer.email)
print(answer.company_info)

teddy@teddynote.com
테디노트


In [38]:
import os

os.environ["SERPAPI_API_KEY"] = (
    "5eccdcee02695a9daa6175f7fc7b9e055b33aa289e6707f0a8a2694cb07d86c4"
)

In [13]:
# pip install google-search-results

In [39]:
from langchain_community.utilities import SerpAPIWrapper

params = {"engine": "google", "gl": "kr", "hl": "ko", "num": "3"}

search = SerpAPIWrapper(params=params)

In [40]:
answer.company_info

'테디노트'

In [41]:
search_query = f"{answer.person} {answer.company_info} {answer.email}"
search_query

'테디 테디노트 teddy@teddynote.com'

In [42]:
search_result = eval(search.run(search_query))

In [24]:
type(search_result)

list

In [43]:
print(search_result[0])

테디노트 데이터와 인공지능을 좋아하는 개발자 노트 · 검색. 토글 메뉴. 카테고리 · 태그 · 연도 · 강의 · 어바웃미 · Teddy. Creator & Data Lover. 팔로우. Pangyo, ...


In [45]:
search_result_string = "\n".join(search_result)

In [46]:
answer

EmailSummary(person='테디', company_info='테디노트', phone_number='', email='teddy@teddynote.com', subject='RAG 솔루션 시연 관련 미팅 제안', summary='테디노트의 테디가 이은채 대리님에게 RAG 솔루션 시연을 위한 미팅을 제안합니다. 테디노트는 AI 및 RAG 솔루션 분야에서 다양한 경험과 전문성을 가지고 있으며, 귀사의 데이터 활용을 극대화하고 실시간으로 정확한 정보 제공을 통해 비즈니스 의사결정을 지원하는데 탁월한 성능을 보입니다. 미팅은 다음 주 목요일(7월 18일) 오전 10시에 귀사 사무실에서 제안되었으며, 다른 일정이 필요하면 조율 가능합니다.', date='다음 주 목요일(7월 18일) 오전 10시')

In [61]:
from langchain_core.prompts import PromptTemplate

report_prompt = PromptTemplate.from_template(
    """당신은 이메일의 주요 정보를 바탕으로 요약 정리해 주는 전문가 입니다.
당신의 임무는 다음의 이메일 정보를 바탕으로 보고서 형식의 요약을 작성하는 것입니다.
주어진 정보를 기반으로 양식(format)에 맞추어 요약을 작성해 주세요.
답변에는 카테고리별로 emoji를 적극 활용하여 답변해 주세요

#Information:
- Sender: {sender}
- Additional Information about sender: {additional_information}
- Company: {company}
- Email: {email}
- Subject: {subject}
- Summary: {summary}
- Date: {date}

# Format(in markdown format):
🤦 보낸 사람:
- (보낸 사람의 이름, 회사 정보)

📧 이메일 주소:
- (보낸 사람의 이메일 주소)

☕️ 보낸 사람과 관련하여 검색된 추가 정보:
- (검색된 추가 정보)

✅ 주요 내용:
- (이메일 제목, 요약)

📅 일정:
- (미팅 날짜 및 시간)
                                      
#Answer:"""
)

In [62]:
from langchain_core.output_parsers import StrOutputParser

In [63]:
report_chain = (
    report_prompt | ChatOpenAI(model="gpt-4-turbo", temperature=0) | StrOutputParser()
)

In [64]:
report_response = report_chain.invoke(
    {
        "sender": answer.person,
        "additional_information": search_result_string,
        "company": answer.company_info,
        "email": answer.email,
        "subject": answer.subject,
        "summary": answer.summary,
        "date": answer.date,
    }
)

In [65]:
print(report_response)

🤦 보낸 사람:
- 테디, 테디노트

📧 이메일 주소:
- teddy@teddynote.com

☕️ 보낸 사람과 관련하여 검색된 추가 정보:
- 테디노트는 데이터 분석, 머신러닝, 딥러닝, LLM에 대한 내용을 다루며, 연구보다는 개발에 관심이 많은 테디가 운영합니다. 테디노트는 다양한 데이터 및 AI 관련 교육 콘텐츠를 제공하며, 특히 RAG 솔루션에 대한 전문성을 보유하고 있습니다.

✅ 주요 내용:
- 제목: RAG 솔루션 시연 관련 미팅 제안
- 요약: 테디노트의 테디가 이은채 대리님에게 RAG 솔루션 시연을 위한 미팅을 제안합니다. 테디노트는 AI 및 RAG 솔루션 분야에서 다양한 경험과 전문성을 가지고 있으며, 귀사의 데이터 활용을 극대화하고 실시간으로 정확한 정보 제공을 통해 비즈니스 의사결정을 지원하는데 탁월한 성능을 보입니다. 미팅은 다음 주 목요일(7월 18일) 오전 10시에 귀사 사무실에서 제안되었으며, 다른 일정이 필요하면 조율 가능합니다.

📅 일정:
- 다음 주 목요일(7월 18일) 오전 10시
